In [1]:
import platform
import sys
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import functions.preprocessing as prep

import sklearn
from sklearn import svm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pyltr
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("This jupyter notebook is running on Python " + platform.python_version())
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6)  # Tested with Python==3.7.9

This jupyter notebook is running on Python 3.7.9


In [2]:
training_set_file_path = 'data/training_set_VU_DM.csv'
test_set_file_path = 'data/test_set_VU_DM.csv'
training_data = pd.read_csv(training_set_file_path)
test_data = pd.read_csv(test_set_file_path)

training_data = prep.parseDateTime(training_data)
training_data = prep.replaceNaN(training_data)
training_data = prep.mergeComps(training_data)

test_data = prep.parseDateTime(test_data)
test_data = prep.replaceNaN(test_data)
test_data = prep.mergeComps(test_data)


c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return dataFrame.drop('date_time', 1)
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  'comp2_rate_percent_diff', 'comp5_rate_percent_diff', 'comp8_rate_percent_diff'], 1)


In [3]:
conditions = [
    (training_data['booking_bool'] == 1),
    (training_data['click_bool'] == 1 ) & (training_data['booking_bool'] == 0),
    (training_data['click_bool'] == 0),
    ]
values = ['5', '1', '0']
training_data['relevancy'] = np.select(conditions, values)
training_data['relevancy'] = training_data['relevancy'].astype('category')

In [4]:
training_data_y = training_data.relevancy
search_id = training_data.srch_id.tolist()
splitNumber = int(len(training_data_y)*0.8)

In [5]:
columns = ['site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_starrating', 'prop_brand_bool', 'prop_location_score1',
                              'prop_log_historical_price', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
                              'srch_saturday_night_bool', 'random_bool', 'comp_rate', 'comp_inv', 'comp_rate_percent_diff']
training_data_x = training_data[columns]
test_data = test_data[columns]

In [6]:
val_data_x = training_data_x[splitNumber:]
val_data_y = training_data_y[splitNumber:]
val_search_id = search_id[splitNumber:]
training_data_x = training_data_x[:splitNumber]
training_data_y = training_data_y[:splitNumber]
training_search_id = search_id[:splitNumber]

In [7]:
scaler = StandardScaler().fit(training_data_x)
training_data_x = scaler.transform(training_data_x)
scaler = StandardScaler().fit(val_data_x)
val_data_x = scaler.transform(val_data_x)

In [12]:
metric = pyltr.metrics.NDCG(k=5)

monitor = pyltr.models.monitors.ValidationMonitor(val_data_x, val_data_y, val_search_id, metric=metric,
                                                      stop_after=100)
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=1000,
    max_features=0.5,
    learning_rate=0.02,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)
model.fit(training_data_x, training_data_y, training_search_id, monitor=monitor)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 


TypeError: __init__() got an unexpected keyword argument 'presort'